In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import shutil
from time import sleep

In [2]:
api_key = 'YOUR_API_KEY_HERE' # input api key to be able to run this notebook

This section returns a dataframe of the search results from flicker. Just change the string for the search_terms assignment and pages.

In [175]:
result = []
search_terms = 'shattered window'.replace(' ', '+')
pages = (21, 30)

for i in range(pages[0], pages[1]):

    url = f'https://www.flickr.com/services/rest/?method=flickr.photos.search&api_key={api_key}&text={search_terms}&format=json&nojsoncallback=1&page={i}'

    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'lxml')
    res = json.loads(soup.find('p').text)
    res = res['photos']['photo']
    result+=res
    sleep(1)
    
result = pd.DataFrame(result)

This section appends the images to a list from the above search results.

In [176]:
name = []
img = []

for i in range(len(result)):
    server = result['server'][i]
    secret = result['secret'][i]
    id = result['id'][i]

    name.append(f'{id}_{secret}_n.jpg')
    img.append(f'https://live.staticflickr.com/{server}/{name[i]}')

This section saves the images to a folder called flickr. It also saves the search results to a csv.

In [178]:
for i in range(len(img)):
    sleep(1)
    
    r = requests.get(img[i], stream = True)
    r.raw.decode_content = True

    with open(f'./flickr/{name[i]}','wb') as f:
        shutil.copyfileobj(r.raw, f)

In [179]:
csv_name = search_terms.replace('+', '_')
result.to_csv(f"./flickr/{csv_name}.csv", index=False)